<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/total_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualising the imperceptible: total flow

this is a notebook which is used to have a set of images move through the processing steps which can individually be found in the topic related folders.

A **batch** of images will be processed and all intermediate steps will be saved. This is not an optimized flow, yet a flow allowing the user to intervene where necessary and have updated images continue throughout the flow.

This flow is created to test with some experts and capture their feedback, it is not intended for day-to-day usage. One testing day will be organized in which this notebook will be used with some unique images.

## steps

1. Sensor dust removal
2. Image Editing (LaMa)
3. Background Removal
4. Background Recoloring: not yet
5. Clothes recoloring: not yet
6. Skin retouching
<!-- 7. Face Detection -->
7. optional: Color Corrections: not yet
8. Color Grading: 
9. Image upscaling

In [1]:
!nvidia-smi

Tue Jun 14 10:10:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# first we'll link a database connection:
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py --silent
from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

what's the username for nextcloud? colab
what's the password for user colab? ··········
0
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  


## SETUP

we'll link this instance of the machine learning flow to your name:

In [3]:
#@title setting up the next cloud folders
tname = 'total' #@param {type:"string"}
if(tname=='total'):
    print("Are you sure you don't want to change the name?")

# make input dynamic with tname
#input_tname = '/content/database/' + tname + '/input'
input_step1, output_step1 = create_io(database=nextcloud,topic=tname,library='step1_sensor_dust', input_redirect='/content/database/total/input')
input_step2, output_step2 = create_io(database=nextcloud,topic=tname,library='step2_lama', input_redirect=output_step1)
input_step3, output_step3 = create_io(database=nextcloud,topic=tname,library='step3_bg_removal', input_redirect=output_step2)
input_step4, output_step4 = create_io(database=nextcloud,topic=tname,library='step4_clothes_coloring', input_redirect=output_step3)
input_step5, output_step5 = create_io(database=nextcloud,topic=tname,library='step5_skin_retouch', input_redirect=output_step4)
input_step6, output_step6 = create_io(database=nextcloud,topic=tname,library='step6_color_corrections', input_redirect=output_step5)
input_step7, output_step7 = create_io(database=nextcloud,topic=tname,library='step7_color_grading', input_redirect=output_step6)
input_step8, output_step8 = create_io(database=nextcloud,topic=tname,library='step8_image_upscaling', input_redirect=output_step7)
input_step9, output_step9 = create_io(database=nextcloud,topic=tname,library='step9_noise_addition', input_redirect=output_step8)

Are you sure you don't want to change the name?


## Step 1: Sensor dust removal

we'll link the main input folder and write the output images in the output folder of step 1.

In [4]:
#@title imports of libraries & setting up
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os, sys
!curl https://raw.githubusercontent.com/Tschucker/Python-Automatic-Sensor-Dust-Removal/main/shapedetector.py -o /content/shapedetector.py
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
from shapedetector import ShapeDetector
import imutils
from google.colab.patches import cv2_imshow

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1255  100  1255    0     0   7171      0 --:--:-- --:--:-- --:--:--  7171


In [5]:
#@title Set inpainting options and run the model
radius = 11 #@param {type:"slider",min:1, max:50}
flags = cv2.INPAINT_TELEA #@param ["cv2.INPAINT_TELEA","cv2.INPAINT_NS"]

def inpaint_img(img_path, img_name, output_path, radius=10, flags=cv2.INPAINT_TELEA):
  #color version
  cimg = cv2.imread(img_path)
  #grey scale image
  img = cv2.imread(img_path,0)

  #Apply Global Threshold
  m = np.mean(img, dtype=int)
  global_thresh = cv2.threshold(img,int(m/1.2),255,cv2.THRESH_BINARY_INV)[1]

  #Perform Adaptive Threshold
  adaptive_thresh_img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,19,10)

  #Image Magnification Filter Kernel
  KERNEL = np.ones((10,10), dtype=int)*10

  #Filter the thresholded images*
  img_filt = cv2.filter2D(adaptive_thresh_img,-1,KERNEL)
  #global_thresh = cv2.filter2D(global_thresh,-1,KERNEL)

  #Apply multiple times
  for i in range(2):
      KERNEL_i = np.ones((int(10),int(10)), dtype=int)*10
      img_filt = cv2.filter2D(img_filt,-1,KERNEL_i)

  #Combine Thresholds
  comb = img_filt + global_thresh

  #Find and Classify Contours of Image
  cnts = cv2.findContours(comb.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  sd = ShapeDetector()
  cimg_copy = cimg.copy()
  for c in cnts:
      # compute the center of the contour, then detect the name of the
      # shape using only the contour
      M = cv2.moments(c)
      if M["m00"] != 0:
          cX = int((M["m10"] / M["m00"]) * 1)
          cY = int((M["m01"] / M["m00"]) * 1)
          shape = sd.detect(c)
          # multiply the contour (x, y)-coordinates by the resize ratio,
          # then draw the contours and the name of the shape on the image
          if len(c) < 50:
              c = c.astype("float")
              c *= 1
              c = c.astype("int")
              cv2.drawContours(cimg_copy, [c], -1, (0, 255, 0), 2)
              cv2.putText(cimg_copy, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
  
  #Create Dust Mask
  img_mask = np.zeros((img.shape[0], img.shape[1]), dtype='uint8')
  for c in cnts:
      # compute the center of the contour, then detect the name of the
      # shape using only the contour
      M = cv2.moments(c)
      if M["m00"] != 0:
          cX = int((M["m10"] / M["m00"]) * 1)
          cY = int((M["m01"] / M["m00"]) * 1)
          shape = sd.detect(c)
          # multiply the contour (x, y)-coordinates by the resize ratio,
          # then draw the contours and the name of the shape on the image
          if len(c) < 50:
              c = c.astype("float")
              c *= 1
              c = c.astype("int")
              cv2.fillPoly(img_mask, pts=[c], color=(255,255,255))

    
  #Inpaint the image
  cimg_inpaint = cv2.inpaint(cimg, img_mask, radius, flags=flags)

  #Show and Save Final Image
  save_img_pth = os.path.join(output_path,img_name)
  cv2.imwrite(save_img_pth, cimg_inpaint)

  # plt_out = cv2.cvtColor(cimg_inpaint, cv2.COLOR_BGR2RGB)
  # return plt_out

for img_name in os.listdir(input_step1):
  if('.jpg' in img_name or '.png' in img_name or '.jpeg' in img_name):
    print("processing ",img_name)
    img_path = os.path.join(input_step1,img_name)
    inpaint_img(img_path, img_name, output_step1, radius=radius, flags=flags)

processing  01.jpg
processing  02.jpg
processing  03.jpg
processing  04.jpg
processing  05.jpg
processing  06.jpg
processing  07.jpg
processing  08.jpg
processing  09.jpg
processing  10.jpg


### verification

Now it's time to go to the database and verify the results. If needed we adapt the images locally. 

## step 2: Minor retouching: image editing LaMa 
Once verified we want to continue with the next step, being image editing (LaMa model)

therefore we link the output folder of previous step to the inputfolder of this step.

In [10]:
#@title imports of libraries & setting up
root_path2 = '/content/lama'

# clone the repository
if not os.path.exists(root_path2):
  !git clone https://github.com/saic-mdal/lama {root_path2}
# Set up the environment
print('\n> Install dependencies')
!pip install -q -r lama/requirements.txt 
!pip install torchtext==0.9.0 --quiet 
!pip install torchvision==0.9.0 --quiet 
!pip install -q wget 

# model path
model_path = os.path.join(nextcloud,'ImageEditing/big-lama')

# fixing openCV
print('>fixing opencv')
!pip uninstall -q opencv-python-headless -y 
!pip install -q opencv-python-headless==4.1.2.30 
!pip install torch==1.8.1


> Install dependencies
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.

> Download the model

> Unzip the model
>fixing opencv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl (804.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.9.0 requires torch==1.8.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.9.0 requires torch==1.8.0, but you have torch 1.8.1 whi

In [11]:
#@title imports & helper functions

import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil

canvas_html = """
<style>
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  position: absolute;
  padding: 15px 25px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}
</style>
<canvas1 width=%d height=%d>
</canvas1>
<canvas width=%d height=%d>
</canvas>

<button class="button">Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')

var canvas1 = document.querySelector('canvas1')
var ctx1 = canvas.getContext('2d')


ctx.strokeStyle = 'red';

var img = new Image();
img.src = "data:image/%s;charset=utf-8;base64,%s";
console.log(img)
img.onload = function() {
  ctx1.drawImage(img, 0, 0);
};
img.crossOrigin = 'Anonymous';

ctx.clearRect(0, 0, canvas.width, canvas.height);

ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
    button.remove()
    canvas.remove()
  }
})
</script>
"""

def draw(imgm, filename='drawing.png', w=400, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, w,h, filename.split('.')[-1], imgm, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)


step2_temp = os.path.join(nextcloud, tname, "step2temp")

os.makedirs(step2_temp,exist_ok=True)

for i in os.listdir(input_step2):
  shutil.copy2(os.path.join(input_step2,i),step2_temp)

Minor Retouching via LaMa

In [12]:
#@title create masks
%cd {root_path2}
#todo still need to fix the layout of the button

for i in os.listdir(step2_temp):
  if('mask' in i):
    # fix if we run the cell multiple times, not to take into account the masks
    continue
  fpath = os.path.join(step2_temp,i)

  image64 = base64.b64encode(open(fpath, 'rb').read())
  image64 = image64.decode('utf-8')
  fname = fpath.split('/')[-1].split('.')[0]
  img = np.array(plt.imread(f'{fpath}')[:,:,:3])

  draw(image64, filename=f"./{fname}_mask.png", w=img.shape[1], h=img.shape[0], line_width=0.04*img.shape[1])
  
for i in os.listdir(step2_temp):
  if('mask' in i):
    # fix if we run the cell multiple times, not to take into account the masks
    continue
  fpath = os.path.join(step2_temp,i)
  fname = fpath.split('/')[-1].split('.')[0]
  with_mask = np.array(plt.imread(f"./{fname}_mask.png")[:,:,:3])
  mask = (with_mask[:,:,0]==1)*(with_mask[:,:,1]==0)*(with_mask[:,:,2]==0)
  plt.imsave(f"{step2_temp}/{fname}_mask.png",mask, cmap='gray')


/content/lama


In [ ]:
#@title Run inpainting
if '.jpeg' in fpath:
  !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py  model.path={model_path} indir={step2_temp}  outdir={output_step2}  dataset.img_suffix=.jpeg   > /dev/null
elif '.jpg' in fpath:
  !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py  model.path={model_path} indir={step2_temp}  outdir={output_step2}  dataset.img_suffix=.jpg    #> /dev/null
elif '.png' in fpath:
  !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py  model.path=$(pwd)/big-lama indir={step2_temp}  outdir={output_step2}  dataset.img_suffix=.png    > /dev/null
else:
  print(f'Error: unknown suffix .{fname.split(".")[-1]} use [.png, .jpeg, .jpg]')

plt.rcParams['figure.dpi'] = 200

for i in (os.listdir(output_step2)):
  i_name = i.replace('_mask','')
  os.rename(os.path.join(output_step2,i),os.path.join(output_step2,i_name))

Detectron v2 is not installed
[2022-06-14 10:40:59,093][saicinpainting.utils][WARNING] - Setting signal 10 handler <function print_traceback_handler at 0x7fe535d619e0>
[2022-06-14 10:41:02,939][root][INFO] - Make training model default
[2022-06-14 10:41:02,939][saicinpainting.training.trainers.base][INFO] - BaseInpaintingTrainingModule init called
[2022-06-14 10:41:02,940][root][INFO] - Make generator ffc_resnet
[2022-06-14 10:41:03,473][saicinpainting.training.trainers.base][INFO] - Generator
FFCResNetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): FFC_BN_ACT(
      (ffc): FFC(
        (convl2l): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1), bias=False, padding_mode=reflect)
        (convl2g): Identity()
        (convg2l): Identity()
        (convg2g): Identity()
        (gate): Identity()
      )
      (bn_l): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn_g): Identity()
      (act_l): ReLU(inplace=True)


## Step 3: background removal

Again, verify the outcome of step 2. 

Now we'll subtract the background using the PaddleSeg model



In [ ]:
#@title imports for paddleseg
!pip install -q PaddlePaddle
root_path = '/content/PaddleSeg'

# clone the repository
if not os.path.exists(root_path):
  !git clone https://github.com/PaddlePaddle/PaddleSeg {root_path}

%cd {root_path}
!pip -qq install -r requirements.txt'
!pip install -e .

# installing Matting
%cd Matting
!pip -qq install -r requirements.txt

In [ ]:
#@title downloading the models
# download model checkpoint 
model_path = root_path + '/Matting/data/model'
model_params = 'https://paddleseg.bj.bcebos.com/matting/models/human_matting-resnet34_vd.pdparams'
model_inf = 'https://paddleseg.bj.bcebos.com/matting/models/deploy/pp-humanmatting-resnet34_vd.zip'
# make folders
os.makedirs(model_path, exist_ok=True)
if not os.path.exists(os.path.join(model_path,'human_matting.pdparams')):
  print('\n> Download the model params')
  !curl {model_params} -o {os.path.join(model_path,'human_matting.pdparams')}
else:
  print ('\n> File already downloaded')
if not os.path.exists(os.path.join(model_path,'human_matting-resnet.zip')):
  print('\n> Download the model')
  !curl {model_inf} -o {os.path.join(model_path,'human_matting-resnet.zip')}
  !unzip -q {os.path.join(model_path,'human_matting-resnet.zip')} -d {os.path.join(model_path)}
else:
  print ('\n> File already downloaded')

In [ ]:
mode = 'background_replacement' #@param ['background_removal','background_replacement']

step3_temp = os.path.join(nextcloud, tname, 'step3temp')
os.makedirs(step3_temp, exist_ok=True)

if(mode=='background_removal'):
  # predict
  !export CUDA_VISIBLE_DEVICES=0

  !python predict.py \
      --config configs/human_matting/human_matting-resnet34_vd.yml \
      --model_path data/model/human_matting.pdparams \
      --image_path {input_step3} \
      --save_dir {step3_temp} \
      --fg_estimate True

  for i in os.listdir(step3_temp):
    if('_alpha' in i):
      #we only want to copy the output files to the next step
      continue

    i_name = i.replace('_clip','')

    shutil.copy2(os.path.join(step3_temp,i),os.path.join(output_step3,i_name))
else:
  # bg replacement
  #@markdown either choose an rgbw value as background or type the path to the bgimage:

  background = 'w' #@param ['r','g','b','w'] {allow-input: true}

  !export CUDA_VISIBLE_DEVICES=0

  for infer_img in os.listdir(input_step3):

    !python bg_replace.py \
        --config configs/human_matting/human_matting-resnet34_vd.yml \
        --model_path data/model/human_matting.pdparams \
        --image_path {os.path.join(input_step3,infer_img)} \
        --save_dir {step3_temp} \
        --background {background} \
        --fg_estimate True

  for i in os.listdir(step3_temp):
    if('_' in i):
      #we only want to copy the output files to the next step
      continue

    shutil.copy2(os.path.join(step3_temp,i),os.path.join(output_step3,i))

## Step 4: Clothes recoloring

In [ ]:
import shutil
#@markdown as we're not implementing clothes recoloring yet, we copy the folders to surpass this step
for file in os.listdir(input_step4):
  shutil.copy2(os.path.join(input_step4, file), output_step4)

## Step 5: Skin retouching

we'll implement the retouchML library

In [ ]:
#@title install models and prerequisites
#@markdown, ignore the warning messages
!pip install 'h5py==2.10.0' 
# numpy versions from 1.20 throw an error further down the road, might need to restart the runtime.
!pip install numpy==1.19.5
root_path5 = '/content/retouchML'
# clone the repository
if not os.path.exists(root_path5):
  !git clone https://github.com/ju-leon/RetouchML {root_path5}

%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
#@title we first create the 'improved' images of faces
#@markdown this might take a while

%cd {root_path5}
# make folders
os.makedirs('aligned_images',exist_ok=True)
os.makedirs('alignement_vector',exist_ok=True)

!python align_images.py {input_step5} aligned_images/ alignement_vector/

!python encode_images.py aligned_images/ generated_images/ latent_representations/ \
    --vgg_url=https://rolux.org/media/stylegan/vgg16_zhang_perceptual.pkl \
    --lr=0.4 \
    --iterations=200 \
    --use_best_loss=True \
    --early_stopping=True \
    --load_resnet=True \
    --composite_blur=6 # default=8


In [ ]:
#@title then we stitch them back together with their original image
face_path = "generated_images/" 
mask_path = "masks/"
vector_path = "alignement_vector/"
for img_name in os.listdir(input_step5):
  raw_path = os.path.join(input_step5, img_name)
  out_path = os.path.join(output_step5, img_name)

  !python fit_faces.py {raw_path} {face_path} {mask_path} {vector_path} {out_path}

## Step 6: Color Corrections

Enhancement of the colors (CURL)



In [ ]:
root_path6 = '/content/CURL'

# clone the repository
if not os.path.exists(root_path6):
  !git clone https://github.com/sjmoran/CURL {root_path6}
%cd {root_path6}


In [ ]:
#@title imports
import numpy as np
from PIL import Image
import sys
import torch
import torchvision.transforms.functional as TF
import requests
from io import BytesIO
import matplotlib.pyplot as plt

# Imports from the code written by authors inside modules
import model
import util
from util import ImageProcessing

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # Might not work without GPU so if you want the cpu verson, clone https://github.com/deshwalmahesh/CURL---cpu-gpu

In [ ]:
#@title Helper functions
def resize(image, new_width_height = 1920, convert_RGB = True):
  '''
  Resize and return Given Image
  args:
    path: Image Path, BytesIO or the image 
    new_width_height = Reshaped image's width and height. # If integer is given, it'll keep the aspect ratio as it is by shrinking the Bigger dimension (width or height) to the max of new_width_height  and then shring the smaller dimension accordingly 
    convert_RGB: Whether to Convert the RGBA image to RGB (by default backgroud is white)
  '''
  image = Image.open(image) if isinstance(image, (str, BytesIO)) else image
  w, h = image.size

  fixed_size = new_width_height if isinstance(new_width_height, int) else False

  if fixed_size:
    if h > w:
      fixed_height = fixed_size
      height_percent = (fixed_height / float(h))
      width_size = int((float(w) * float(height_percent)))
      image = image.resize((width_size, fixed_height), Image.NEAREST)

    else:
      fixed_width = fixed_size
      width_percent = (fixed_width / float(w))
      height_size = int((float(h) * float(width_percent)))
      image = image.resize((fixed_width, height_size), Image.NEAREST) # Try Image.ANTIALIAS inplace of Image.NEAREST

  else:
    image = image.resize(new_width_height)

  if image.mode == "RGBA" and convert_RGB:
  
    new = Image.new("RGBA", image.size, "WHITE") # Create a white rgba background
    new.paste(image, (0, 0), image) # Paste the image on the background.
    image = new.convert('RGB')

  return image



def load_image(path, resize_image_size = 1920):
    '''
    Load the image as tensor according to the format authors have used in the code
    '''
    if ("https" in path) or ("http" in path):
      image = Image.open(BytesIO(requests.get(path).content))

    else:
      image = Image.open(path)

    if image.mode != 'RGB':
      image = image.convert('RGB')
    
    if resize:
      image = resize(image, resize_image_size)
               
    return TF.to_tensor(image).to(DEVICE)

In [ ]:
#@title load pre)trained model
checkpoint_filepath = "./pretrained_models/adobe_dpe/curl_validpsnr_23.073045286204017_validloss_0.0701291635632515_testpsnr_23.584083321292365_testloss_0.061363041400909424_epoch_510_model.pt"

# Build Model
net = model.CURLNet()
checkpoint = torch.load(checkpoint_filepath, map_location=DEVICE)
net.load_state_dict(checkpoint['model_state_dict'])
net.eval()
if DEVICE == 'cuda':
  net.cuda()


def evaluate(img, convert_uint = False):
    """
    Evaluate the model per image instance. Image of Batch size 1. Can be used in API production
    """
    img = load_image(img)

    with torch.no_grad():

        img = img.unsqueeze(0)
        img = torch.clamp(img, 0, 1)

        net_output_img_example , _ = net(img)

        net_output_img_example_numpy = net_output_img_example.squeeze(0).data.cpu().numpy()
        net_output_img_example_numpy = ImageProcessing.swapimdims_3HW_HW3(net_output_img_example_numpy)
        return (net_output_img_example_numpy* 255).astype(np.uint8) if convert_uint else net_output_img_example_numpy

In [ ]:
imgs_to_convert = os.listdir(input_step6)
# for _img in imgs_to_convert:
#     # Load .png image
#   image = cv2.imread(os.path.join(input_step6,_img))
#   # Save .jpg image
#   cv2.imwrite(os.path.join(output_step6,'{}.jpg').format(_img.split('.')[0]), image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
# imgs_to_convert = os.listdir(input_step6)

for _img in imgs_to_convert:
  print(_img)
  result = evaluate(os.path.join(input_step6,_img), convert_uint = True) # gives you array between 0-1 so if you want an "Image", use 'convert_uint = True', then Image.fromarray(array).save(path)
  Image.fromarray(result).save(os.path.join(output_step6,'{}.jpg'.format(_img.split('.')[0])))

## Step 7: Color Grading

Model: deep preset

In [ ]:
#@title install model and prerequisites
import os
root_path7 = '/content/DeepPreset'

# folder with style transfer
style_folder = '/content/database/colorGrading/style/pureColor'

# clone the repository
if not os.path.exists(root_path7):
  !git clone https://github.com/minhmanho/deep_preset {root_path7}

!pip -q install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html


In [ ]:
#@title run the colorgrading
%cd {root_path7}

source = '/content/database/colorGrading/DP_model/dp_wPPL.pth.tar'

!CUDA_VISIBLE_DEVICES=0 python run.py \
     --content {input_step7} \
     --style {style_folder} \
     --out {output_step7} \
     --ckpt {source} \
     --size 400x592

## Step 8: image upscaling

Model: RealESRGan

[ESRGan](https://github.com/luca-arts/seeingtheimperceptible/blob/main/notebooks/basicSuperRestoration/tests/Real_ESRGAN.ipynb)


In [ ]:
#@title setup and clone git repo
import os
root_path8 = '/content/BasicSR'

# clone the repository
if not os.path.exists(root_path8):
  !git clone https://github.com/xinntao/Real-ESRGAN {root_path8}

%ls

In [ ]:
#@title install model and prerequisites
%cd {root_path8}

# Set up the environment
!pip install -q basicsr
!pip install -q facexlib
!pip install -q gfpgan
!pip install -q -r requirements.txt
!python setup.py develop

# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P /content/BasicSR/experiments/pretrained_models


In [ ]:
#@title upscale the images
scale=2 #@param {type:"slider", min:1, max:5, step:0.5}
!python inference_realesrgan.py -n RealESRGAN_x4plus -i {input_step8} -o {output_step8} --outscale 3.5 --face_enhance

# Step 9 add some noise

In [ ]:
#@title imports
# installing the needed libs
print ('\n> Installing OpenCV')
!pip install opencv-python

print ('\n> Installing Numpy')
!pip install numpy

import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import os

In [ ]:
#@title helper functions
# functions for different noise-types
def normalize(mask):
    return (mask - mask.min()) / (mask.max() - mask.min())

def add_noise(noise_fun, gray: bool=False, **kwargs):
    img = kwargs.get('img')
    image = np.array(img, dtype=float)
    noise = noise_fun(**kwargs)
    if(gray):
      gray_ch = cv2.cvtColor(noise.astype(np.float32), cv2.COLOR_BGR2GRAY)
      #change color noise to gray noise for each channel
      noise = cv2.merge([gray_ch,gray_ch,gray_ch])
    image_out = image + noise
    image_out = np.uint8(normalize(image_out) * 255)
    return image_out

def gaussian_noise(**kwargs):
    mu=kwargs.get('mu')
    sigma=kwargs.get('sigma')
    image=kwargs.get('img')
    noise = np.random.normal(mu, sigma, image.shape)
    return noise

def rayleigh_noise(**kwargs):
    a = kwargs.get('a')
    image = kwargs.get('img')
    noise = np.random.rayleigh(a, size=image.shape)
    return noise

def gamma_noise(**kwargs):
    scale = kwargs.get('scale')
    image = kwargs.get('img')
    noise = np.random.gamma(shape=1, scale=scale, size=image.shape)
    return noise

def exponent_noise(**kwargs):
    scale = kwargs.get('scale')
    image = kwargs.get('img')
    noise = np.random.exponential(scale=scale, size=image.shape)
    return noise

def average_noise(**kwargs):
    mean = kwargs.get('mu')
    sigma = kwargs.get('sigma')
    image = kwargs.get('img')
    a = 2 * mean - np.sqrt(12 * sigma)
    b = 2 * mean + np.sqrt(12 * sigma)
    noise = np.random.uniform(a, b, image.shape)
    return noise

def add_gaussian_noise(img, mu=0, sigma=0.1, gray=False):
    img_out = add_noise(gaussian_noise, gray=gray, img=img, mu=mu, sigma=sigma)
    return img_out

def add_rayleigh_noise(img, a=15, gray=False):
    img_out = add_noise(rayleigh_noise,img=img,a=a,gray=gray) 
    return img_out

def add_gamma_noise(img, scale=1, gray=False):
    img_out = add_noise(gamma_noise, img=img, scale=scale,gray=gray)
    return img_out

def add_exponent_noise(img, scale=1.0, gray=False):
    img_out = add_noise(exponent_noise, img=img, scale=scale,gray=gray)
    return img_out

def add_average_noise(img, mean=0, sigma=100, gray=False):
    img_out = add_noise(average_noise, img=img, mu=mu, sigma=sigma, gray=gray)
    return img_out

In [ ]:
noise_type = "gaussian" #@param ["gaussian", "rayleigh", "gamma","exponent","average"]

def use_noise(noise_type, img, mu=0,sigma=5,a=15,scale=1.0,gray=False):
    if(noise_type=="gaussian"):
      img = add_gaussian_noise(img=img, mu=mu, sigma=sigma, gray=gray)
    if(noise_type=="rayleigh"):
       img = add_rayleigh_noise(img=img,a=a,gray=gray)
    if(noise_type=="gamma"):
       img = add_gamma_noise(img=img,scale=scale,gray=gray)
    if(noise_type=="exponent"):
       img = add_exponent_noise(img=img,scale=scale, gray=gray)
    if(noise_type=="average"):
       img = add_average_noise(img=img, mu=mu, sigma=sigma, gray=gray)
    return img

#@markdown mu, sigma for gaussian and average noise
mu = 0.35 #@param {type:"slider", min:0, max:1, step:0.05}
sigma = 16.5 #@param {type:"slider", min:0, max:20, step:0.5}

#@markdown a is for rayleigh noise
a = 10 #@param {type:"slider", min:0, max:20, step:0.5}

#@markdown scale is for gamma and exponent noise
scale = 16.5 #@param {type:"slider", min:0, max:20, step:0.5}

#@markdown an option to use **gray** noise instead of RGB noise
gray = True #@param {type:"boolean"}


#@markdown you can use the noise generation via: `gen_img = use_noise(noise_type, img=_img, mu=mu, sigma=sigma, a=a,scale=scale,gray=gray)`

In [ ]:
#@title execute noise generation
for i in os.listdir(input_step9):
    _img_pth = os.path.join(input_step9,i)
    _img = cv2.imread(_img_pth)
    gen_img = use_noise(noise_type, img=_img, mu=mu, sigma=sigma, a=a,scale=scale,gray=gray)
    cv2.imwrite(os.path.join(output_step9,i),gen_img)

In [ ]:
def read_first_img_in_path(path):
  import os
  from PIL import Image
  img = os.path.join(path,os.listdir(path)[0])
  img = Image.open(img)
  return img
gif_list = list()
gif_list.append(read_first_img_in_path(output_step1))
gif_list.append(read_first_img_in_path(output_step2))
gif_list.append(read_first_img_in_path(output_step3))
gif_list.append(read_first_img_in_path(output_step4))
gif_list.append(read_first_img_in_path(output_step5))
gif_list.append(read_first_img_in_path(output_step6))
gif_list.append(read_first_img_in_path(output_step7))
gif_list.append(read_first_img_in_path(output_step8))
gif_list.append(read_first_img_in_path(output_step9))

gif_list[0].save('/content/database/{}/result.gif'.format(tname),save_all=True, append_images=gif_list[1:], optimize=False, duration=500, loop=0)

# the end

Now we've run through an entire flow, any feedback?